In [2]:
import findspark
findspark.init("/usr/local/Cellar/apache-spark/3.1.2/libexec")
import pyspark
from pyspark.sql import SparkSession

from pyspark.sql.functions import*
from pyspark.sql.types import*
import time

import os

import numpy as np
import re
import pandas as pd 
import traceback
import sys
from time import sleep
import traceback
import smtplib


In [3]:
spark = SparkSession.builder.getOrCreate()

In [4]:
try:
    server = smtplib.SMTP('smtp.gmail.com', 587)
    server.ehlo()
except:
    print('Something went wrong...')

In [5]:
########## Function to Send Email #########

def sendmail(sub, body):
    try:
       SERVER = "us.relay.ibm.com"
       #SERVER = "smtp.gmail.com"
       #SERVER = smtplib.SMTP('smtp.gmail.com', 587)
       #SERVER.ehlo()
       FROM = "romdbload@radial.com"
       TO = ["mlakshm@us.ibm.com"] # must be a list
       SUBJECT = sub
       TEXT = body
       message = 'From:'+FROM+\
       '\nTo: '+", ".join(TO)+\
       '\nSubject: '+SUBJECT+'\n'+TEXT
       print(message)
       server = smtplib.SMTP(SERVER)
       server.sendmail(FROM, TO, message)
       server.quit()
       return 0
    except Exception as e:
       exp_tb=traceback.format_exc()
       print(exp_tb)


In [6]:
sendmail("Test","Test")

From:romdbload@radial.com
To: mlakshm@us.ibm.com
Subject: Test
Test


0

In [7]:
tbllist=[]
jsonpathlists=[]
colslist=[]

with open('json_column_map.txt') as f:
    contents = f.read()
    contents=contents.strip()
    maplist=contents.split("TABLE_NAME:")
    #print(maplist)
    #maplist = list(filter([], maplist))
    for lmap in maplist:
        lmaplist=lmap.split("\n")
        #lmaplist = list(filter(None, lmaplist))
        tbl=lmaplist.pop(0)
        tbllist.append(tbl)
        collist=[]
        jsonpathlist=[]
        for col_json in lmaplist:
            coljsonarr=col_json.split(",")
            if (len(coljsonarr)==2):
               collist.append(coljsonarr[0])
               jsonpathlist.append(coljsonarr[1])
        colslist.append(collist)
        jsonpathlists.append(jsonpathlist)
            
            
f.close()


tbllist = [x for x in tbllist if x != 'KAFKARADIAL.']
tbllist = [x for x in tbllist if x != '']
colslist = [x for x in colslist if x != []]
jsonpathlists = [x for x in jsonpathlists if x != []]


print(tbllist[6])
print(jsonpathlists[6])
print(colslist[6])

KAFKARADIAL.ROM_ORDER_TAX_BREAKUP_STG0
['order.lineItems.0.charges.0.taxes.0.jurisdictionLevel', 'order.id', 'order.lineItems.0.charges.0.taxes.0.situs', 'order.lineItems.0.charges.0.taxes.0.impositionValue', 'order.sellerId', 'order.lineItems.0.charges.0.taxes.0.impositionType', 'order.lineItems.0.charges.0.taxes.0.jurisdictionId']
['JURISDICTION_LEVEL', 'ORDER_ID', 'TAX_SITUS', 'IMPOSITION_VALUE', 'SELLER_ID', 'IMPOSITION_TYPE', 'JURISDICTION_ID']


In [8]:

##### Input Tables ######

#tbllist=['KAFKARADIAL.ORDER_HEADER_STG', 'KAFKARADIAL.ORDER_LINE_STG', 'KAFKARADIAL.ORDER_CHARGES_STG']

In [9]:

##### Column Mapping ######

#jsonpathlists=[['msg.order.orderTotal.promotionCharge', 'msg.order.id', 'msg.order.sellerId', 'msg.order.orderTotal.tax', 'msg.order.orderTotal.total', 'msg.order.orderTotal.adjustmentCharge', 'msg.order.type'], ['msg.order.id', 'msg.order.lineItems.0.quantity', 'msg.order.lineItems.0.lineNo', 'msg.order.lineItems.0.color', 'msg.order.lineItems.0.lineTotal.totalTaxAmount', 'msg.order.lineItems.0.unitPrice', 'msg.order.lineItems.0.lineTotal.totalPromotionChargeAmount', 'msg.order.lineItems.0.lineTotal.netChargeAmount', 'msg.order.lineItems.0.itemDetail.itemId', 'msg.order.lineItems.0.lineTotal.totalAdjustmentChargeAmount', 'msg.order.lineItems.0.itemDetail.itemDesc', 'lookup'], ['msg.order.id', 'msg.order.lineItems.0.charges.0.id', 'msg.order.lineItems.0.lineNo', 'msg.order.lineItems.0.charges.0.detail.amount', 'msg.order.lineItems.0.charges.0.name']]

In [10]:

###### Columns of the Tables #####

#colslist=[['ORDER_PROMO_AMT', 'ORDER_ID', 'SELLER_ID', 'ORDER_TAX_AMT', 'ORDER_TOTAL_AMT', 'ORDER_ADJ_AMT', 'ORDER_TYPE'], ['ORDER_ID', 'ITEM_QTY', 'LINE_NUM', 'COLOR', 'LINE_TAX_AMT', 'ITEM_AMT', 'LINE_PROMO_AMT', 'LINE_TOTAL_AMT', 'ITEM_ID', 'LINE_ADJ_AMT', 'ITEM_DESC', 'COUNTRY_OF_ORIGIN'], ['ORDER_ID', 'CHARGE_ID', 'LINE_NUM', 'CHARGE_AMOUNT', 'CHARGE_NAME']]

In [11]:

########## Reading Input Stream in JSON Format from Kafka ###########

try:

 df = spark \
  .readStream \
  .option("multiLine", True).option("mode", "PERMISSIVE") \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "localhost:9092") \
  .option("subscribe", "test3") \
  .load() \
  .selectExpr("CAST(value AS STRING)")

except Exception as e:
    
    exp_tb=traceback.format_exc()
    sendmail("ERROR",exp_tb)
    print(exp_tb)
    

In [12]:
df.printSchema()

root
 |-- value: string (nullable = true)



In [13]:
###WASHINGTON

jdbcUrl="jdbc:db2://67beb010-4100-48d1-a281-7b5af97791aa.bs2ipa7w0uv9tsomi9ig.databases.appdomain.cloud:31815/bludb:sslConnection=true;"
user="a5406f1c"
password="eK1EhADAU7y3W61T"

In [14]:
###### Lookup Country of Origin #######

key ="'100'"
keyval="SELECT COUNTRY_OF_ORIGIN FROM KAFKARADIAL.PRODUCT_MASTER where ITEM_ID="+key

print(keyval)
dbl = spark.read.format('jdbc').option('url', jdbcUrl).option('dbtable', '('+keyval+') as t').option('user', user).option('password', password).load()
dbl.show()


SELECT COUNTRY_OF_ORIGIN FROM KAFKARADIAL.PRODUCT_MASTER where ITEM_ID='100'
+-----------------+
|COUNTRY_OF_ORIGIN|
+-----------------+
+-----------------+



In [15]:
def writeToSQLWarehouse(df,epochId,tbl):
  print(tbl)
  #print(df)
  df.write.format("jdbc") \
  .mode("append") \
  .option("url", jdbcUrl) \
  .option("dbtable", tbl) \
  .option("user", user) \
  .option("password", password) \
  .save()

In [16]:
def prepMultiRows(df,epochID):
    df.show()
    l = []
    for row in df.rdd.collect():
       x=row[0]
       dfr = spark.read.option("multiline", "true") \
       .json(spark.sparkContext.parallelize([x]))
       dfr.show()
       l.append(dfr)
    print("***")

    return l

In [17]:
def popTables(df, epochId):  #### Bulk Insert
    
 try:  
    
   if df.rdd.isEmpty():
     print(df.rdd.isEmpty)
   else:
    print("before")
    #df=df.withColumnRenamed("from_json(CAST(value AS STRING))", "msg")
    dflist=prepMultiRows(df,epochId)
    print("after")
    
    # Create an empty RDD
    emp_RDD = spark.sparkContext.emptyRDD()
 
    # Create empty schema
    columns = StructType([])
 
    # Create an empty RDD with empty schema
    dfblk = spark.createDataFrame(data = emp_RDD, schema = columns)
    
    i=0
    for jmsg in dflist:
        jmsg.show()
        #print(jsonpathlist)
        print("****")
        if ( jmsg.columns[0] != '_corrupt_record' ):
            if i==0:
                dfblk=jmsg
                i=i+1
            else:
                dfblk.union(jmsg)  
            
        else:
            print("Corrupt Record:")
            jmsg.show()
            
    prepareExpDF(dfblk,epochId)
    
 except Exception as e:
    
    exp_tb=traceback.format_exc()
    sendmail("ERROR",exp_tb)
    print(exp_tb)


In [18]:
def popTables(df, epochId):
    
 try:  
    
   if df.rdd.isEmpty():
     print(df.rdd.isEmpty)
   else:
    print("before")
    #df=df.withColumnRenamed("from_json(CAST(value AS STRING))", "msg")
    dflist=prepMultiRows(df,epochId)
    print("after")
    for jmsg in dflist:
        jmsg.show()
        #print(jsonpathlist)
        print("****")
        if ( jmsg.columns[0] != '_corrupt_record' ):
            prepareExpDF(jmsg,epochId)
        else:
            print("Corrupt Record:")
            jmsg.show()
    
 except Exception as e:
    
    exp_tb=traceback.format_exc()
    sendmail("ERROR",exp_tb)
    print(exp_tb)



In [19]:
lookuptablelist=[["KAFKARADIAL.PRODUCT_MASTER","COUNTRY_OF_ORIGIN"]]

In [20]:
###### Prepare Dataframe and Populate the DB #######

## [[PRODUCT_MASTER,COUNTRY_OF_ORIGIN],[<table2>,<field2>]]

def lookupItemID(lookuptablelist, item_id):
    
    try:
       
     for lookuptable in lookuptablelist:
            
            lookuptblname=lookuptable[0]
            lookuptblfield=lookuptable[1]
            item_id="'"+item_id+"'"
            print(item_id)
            qry='SELECT '+lookuptblfield+' FROM '+lookuptblname+' WHERE ITEM_ID = '+item_id
            print(qry)
            dbl = spark.read.format('jdbc').option('url', jdbcUrl).option('dbtable', '('+qry+') as t').option('user', user).option('password', password).load()
            dbl.show()
            ctry=dbl.collect()[0][0]
            return ctry

       #### Create Dataframe with JSON value list parsed as the data and the Table Column Names as the Schema ####
       #sdf=spark.createDataFrame(data = jsonvallist, schema = romlistfinal1)
    
    
    except Exception as e:
        print("Exception:")
        print(e)
        exp_tb=traceback.format_exc()
        sendmail("ERROR",exp_tb)
        #print(exp_tb)

In [21]:
#ctry=lookupItemID(lookuptablelist,'100')

In [22]:
import builtins
round = getattr(builtins, "round")
from pyspark.sql.functions import lit

def updateDF(sdf,tbl):
    
    sdf.printSchema()
    if tbl=="KAFKARADIAL.ROM_ORDER_LINE_STG0":
       try:
        sdf=sdf.withColumn("IS_ASSOCIATE_DELIVERY", when(col("IS_ASSOCIATE_DELIVERY") == "false",lit("0") )\
        .when(col("IS_ASSOCIATE_DELIVERY") == "true",lit("1")) \
        .otherwise(col("IS_ASSOCIATE_DELIVERY")))
       except Exception as e:
         print(e)
    
    return sdf
    

In [23]:
def explodeArrays(elm,df, elmstr, elmsplitstr):
            
            elmsplitar=elmsplitstr.split(".0.")
            elmsplitarr=elmsplitar[:-1]
            elmstrprep=""
            ctr=0
            elmarrstr=""
            # Example elm: msg.order.lineItems.0.charges.0.id


            for e in elmsplitarr:
                ### examples e: msg.order.lineItems, charges
                esparr=e.split()
                e=esparr[0]
                e=e.replace('"', '')
                
                if ctr==0:
                    elmstrprep=e
                    elmarrstr=elmstrprep
                    
                    elmarrstr=re.sub('[.]','_',elmarrstr)
                    print(e,"**",elmarrstr)
                    ctr=ctr+1
                else:
                   
                   elmarrstr=elmstrprep+"_"+e
                   elmstrprep=elmstrprep+"."+e

                if (elmarrstr not in df.columns):
                   arrchk=df.select(elmstrprep).collect()[0][0]
                   if arrchk:
                      df=df.withColumn(elmarrstr,explode(elmstrprep))
                      df.show() 
                elmstrprep=elmarrstr
                
            return df,elmarrstr,elmsplitar


In [24]:
######### Obtaining values from the incoming JSON Msg ###########

def prepareExpDF(df,epochId):

#### Iterates for every table that we will be populating ####


 for jsonpathlist,romlistfinal1,tbl in zip(jsonpathlists,colslist,tbllist):
    jsonpathwithval=[]
    jsonpathwithnoval=[]
    romlistwithval=[]
    romlistwithnoval=[]
    jsonpathval=[]
    lookuptablelist=[]
  

    ####### Find values for every column of a table #####
    
    elm_up_arr=[]

    for elm,colval in zip(jsonpathlist,romlistfinal1):
      elmstr=elm
      elmsplitstr=elm
      print(elm)
      try: 
        
        ########## If the JSON Path contains arrays, explode every array and obtain the values for the JSON Path 
                 
        if (elm.find(".0.")>-1):
            
            df,elmarrstr,elmsplitar=explodeArrays(elm,df, elmstr, elmsplitstr)
            elm_up=elmarrstr+"."+elmsplitar[-1]
            elm_up_arr.append(elm_up)
            romlistwithval.append(colval)
            print("********")
            #df.show()
            
        else:
            
            elm_up_arr.append(elm)
            romlistwithval.append(colval)
     
      

      except Exception as e:
        print(e)
     
    prepareDF1(df,elm_up_arr,romlistwithval,epochId,tbl)
    

In [30]:
###### Prepare Dataframe and Populate the DB #######

def prepareDF1(df,elm_up_arr,romlistfinal1,epochId,tbl):
    
    try:
         
       print("&&&&&&&&&&")
       df.show()
       elm_up_arr_clean=[]
       rom_list_clean = [] 
       for elm,colval in zip(elm_up_arr,romlistfinal1):
        
           try:
             df.select(elm)
             elm_up_arr_clean.append(elm)
             rom_list_clean.append(colval)  
           except Exception as e:
             print(e)
              
        
       
       cdf=df.select(elm_up_arr_clean)
       cdf.show()
        
       sdf=cdf.toDF(*rom_list_clean)
       sdf.show()
        
       ##### Drop Duplicates ######
       print("Dropping Duplicates")
       sdf=sdf.dropDuplicates()
       sdf.show()
    
       ######## Update Dataframe ########
       print("Updating")
       sdf = updateDF(sdf,tbl)
       sdf.show()
       
       writeToSQLWarehouse(sdf, epochId,tbl)
    
    except Exception as e:
        print("hererrerre")
        print(e)

In [ ]:
def pullCtry(jsonpathval,romlistwithval,colslist,lookupcolumn,lookupkey,tbl):
    
    ############# Pull COUNTRY_OF_ORIGIN ##############
    
    try:
    
      if ( lookupcolumn in colslist ):

          print("Inside*******")
        
          item_id_idx=romlistwithval.index(lookupkey)
          item_id=jsonpathval[item_id_idx]
          print(item_id)
          tableinfo=[tbl,lookupcolumn]
          lookuptablelist.append(tableinfo)
       
    
          ctry=lookupItemID(lookuptablelist, item_id)
          jsonpathval.append(ctry)
          romlistwithval.append(lookupcolumn)
     
    except Exception as e:
          print(e)
        
    
    return jsonpathval, romlistwithval
    
    

In [31]:
####### Writing Steams to DB #######

query1 = df \
    .writeStream \
    .outputMode("append") \
    .option("partition.assignment.strategy", "range")\
    .foreachBatch(popTables)\
    .start()

query1.awaitTermination(10)

<bound method RDD.isEmpty of MapPartitionsRDD[620] at javaToPython at NativeMethodAccessorImpl.java:0>
before
+--------------------+
|               value|
+--------------------+
|{  "topicName" : ...|
+--------------------+

+----------------+--------------------+--------------------+
|customAttributes|               order|           topicName|
+----------------+--------------------+--------------------+
|          {HTTP}|{[{dest_0, 1000 W...|LVS.UAT01.order_c...|
+----------------+--------------------+--------------------+

***
after
+----------------+--------------------+--------------------+
|customAttributes|               order|           topicName|
+----------------+--------------------+--------------------+
|          {HTTP}|{[{dest_0, 1000 W...|LVS.UAT01.order_c...|
+----------------+--------------------+--------------------+

****
order.id
order.sellerId
order.customerInfo.phone
order.customerInfo.customerId
order.customerOrderId
order.type
order.orderClosed
order.orderSource

False

order.id
order.lineItems.0.sizeCode
order.lineItems ** order_lineItems
+----------------+--------------------+--------------------+--------------------+
|customAttributes|               order|           topicName|     order_lineItems|
+----------------+--------------------+--------------------+--------------------+
|          {HTTP}|{[{dest_0, 1000 W...|LVS.UAT01.order_c...|{[], ANY, [{[], P...|
+----------------+--------------------+--------------------+--------------------+

********
order.sellerId
order.lineItems.0.color
order.lineItems ** order_lineItems
********
order.lineItems.0.isAssociateDelivery
order.lineItems ** order_lineItems
********
order.lineItems.0.size
order.lineItems ** order_lineItems
********
order.lineItems.0.colorCode
order.lineItems ** order_lineItems
********
order.lineItems.0.itemDetail.unitOfMeasure
order.lineItems ** order_lineItems
********
order.lineItems.0.itemDetail.itemId
order.lineItems ** order_lineItems
********
order.lineItems.0.itemDetail.gii
order

+-----------+--------+
|         id|sellerId|
+-----------+--------+
|30002217646|    浣熊|
+-----------+--------+

+-----------+---------+
|   ORDER_ID|SELLER_ID|
+-----------+---------+
|30002217646|     浣熊|
+-----------+---------+

Updating
root
 |-- ORDER_ID: string (nullable = true)
 |-- SELLER_ID: string (nullable = true)

+-----------+---------+
|   ORDER_ID|SELLER_ID|
+-----------+---------+
|30002217646|     浣熊|
+-----------+---------+

KAFKARADIAL.ROM_ORDER_PROMOTION_STG0
order.id
order.sellerId
&&&&&&&&&&
+----------------+--------------------+--------------------+--------------------+--------------------+
|customAttributes|               order|           topicName|     order_lineItems|order_paymentMethods|
+----------------+--------------------+--------------------+--------------------+--------------------+
|          {HTTP}|{[{dest_0, 1000 W...|LVS.UAT01.order_c...|{[], ANY, [{[], P...|{145.5, , CREDIT_...|
+----------------+--------------------+--------------------+--------

order.id
order.sellerId
&&&&&&&&&&
+----------------+--------------------+--------------------+--------------------+--------------------+-----------------------+-----------------------------+
|customAttributes|               order|           topicName|     order_lineItems|order_paymentMethods|order_lineItems_charges|order_lineItems_charges_taxes|
+----------------+--------------------+--------------------+--------------------+--------------------+-----------------------+-----------------------------+
|          {HTTP}|{[{dest_0, 1000 W...|LVS.UAT01.order_c...|{[], ANY, [{[], P...|{145.5, , CREDIT_...|   {[], Price, [], {...|         {7.8, Price, [], ...|
|          {HTTP}|{[{dest_0, 1000 W...|LVS.UAT01.order_c...|{[], ANY, [{[], P...|{145.5, , CREDIT_...|   {[], Price, [], {...|         {1.3, Price, [], ...|
|          {HTTP}|{[{dest_0, 1000 W...|LVS.UAT01.order_c...|{[], ANY, [{[], P...|{145.5, , CREDIT_...|   {[], Shipping, []...|         {0.36, Shipping, ...|
|          {HTTP}|{[{de

+-----------+--------+-------------+-----+
|         id|sellerId|       itemId| type|
+-----------+--------+-------------+-----+
|30002217646|    浣熊|75-MH06-MD-BK|SALES|
|30002217646|    浣熊|75-MH06-MD-BK|SALES|
|30002217646|    浣熊|75-MH06-MD-BK|SALES|
|30002217646|    浣熊|75-MH06-MD-BK|SALES|
+-----------+--------+-------------+-----+

+-----------+---------+-------------+----------+
|   ORDER_ID|SELLER_ID|      ITEM_ID|ORDER_TYPE|
+-----------+---------+-------------+----------+
|30002217646|     浣熊|75-MH06-MD-BK|     SALES|
|30002217646|     浣熊|75-MH06-MD-BK|     SALES|
|30002217646|     浣熊|75-MH06-MD-BK|     SALES|
|30002217646|     浣熊|75-MH06-MD-BK|     SALES|
+-----------+---------+-------------+----------+

Updating
root
 |-- ORDER_ID: string (nullable = true)
 |-- SELLER_ID: string (nullable = true)
 |-- ITEM_ID: string (nullable = true)
 |-- ORDER_TYPE: string (nullable = true)

+-----------+---------+-------------+----------+
|   ORDER_ID|SELLER_ID|      ITEM_ID|ORDER_TYPE|
+--

In [32]:
query1.stop()

In [33]:
query1.status

{'message': 'Stopped', 'isDataAvailable': False, 'isTriggerActive': False}